In [9]:
import re, json, time
from selenium import webdriver
import os

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


# Tratamento


In [10]:
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector

In [11]:
import re


import locale

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

base_path = f"./data/{canais[0].split('@')[-1]}/"



def trata_inscritos(col: pd.Series) -> pd.Series:
    col = col.split(" inscritos")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )

    return int(col.replace(" ", "").replace(",", "."))


def trata_visualizacoes(col: pd.Series) -> pd.Series:
    # treat the characters U+00a0

    # if its a numeric value return it
    if type(col) == int or type(col) == float:
        return col

    col = col.replace("\xa0", "")

    col = col.split(" visualizações")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    return int(col.replace(" ", "").replace(".", "").replace(",", "."))


def trata_data(value):
    meses = {
        "jan": "01",
        "fev": "02",
        "mar": "03",
        "abr": "04",
        "mai": "05",
        "jun": "06",
        "jul": "07",
        "ago": "08",
        "set": "09",
        "out": "10",
        "nov": "11",
        "dez": "12",
    }
    series_data = re.sub(r"\. de", "", value)
    # print(series_data)
    series_data = re.search(r"(\d{1,2} de [a-zA-Z]+ \d{4})", series_data).group(0)
    # replace month
    for k, v in meses.items():
        series_data = series_data.replace(k, v)

    # print(series_data)
    series_data = pd.to_datetime(series_data, format="%d de %m %Y", errors="coerce")
    return series_data



videos = pd.read_csv(base_path + "videos.csv")


videos.assign(
    subscribers=lambda df_: df_["subscribers"].apply(trata_inscritos),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    date=lambda df_: df_["date"].apply(trata_data),
    category="RECENT",
).to_parquet(base_path + "videos.parquet")


# Trata Top videos


import pandas as pd


top_videos = pd.read_csv(base_path + "top_videos.csv")


top_videos.assign(
    date=lambda df_: df_["date"].apply(trata_data),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    category="TOP",
).to_parquet(base_path + "top_videos.parquet")


top_videos_silver = pd.read_parquet(base_path + "top_videos.parquet")


# # Trata lifetime


df_lifetime = pd.read_csv(base_path + "lifetime.csv")


df_lifetime.head()



def days_on_youtube(value) -> int:
    import datetime

    today = datetime.date.today()
    return (today - value.date()).days



(
    df_lifetime.assign(
        date_joined=lambda df_: df_["date_joined"].apply(trata_data),
        total_views=lambda df_: df_["total_views"].apply(trata_visualizacoes),
        total_subs=lambda df_: df_["total_subs"].apply(trata_inscritos),
        days_since_joined=lambda df_: df_["date_joined"].apply(days_on_youtube),
    )
).to_csv(base_path + "lifetime_clean.csv", index=False)

In [12]:
# # join


videos = pd.read_parquet(base_path + "videos.parquet")
top_videos = pd.read_parquet(base_path + "top_videos.parquet")


# join the two dataframes

videos = pd.concat([videos, top_videos])


videos_sorted = videos.sort_values("channel_name")

videos_sorted["subscribers"] = videos_sorted["subscribers"].fillna(method="ffill")


videos_sorted = videos_sorted.sort_values(["channel_name", "category", "date"])


videos_sorted.to_parquet(base_path + "videos_sorted.parquet")


pd.read_parquet(base_path + "videos_sorted.parquet").to_csv(
    base_path + "videos_sorted.csv", index=False
)

last 30 days

In [13]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

# print(canais)

base_url = "https://socialblade.com/youtube/channel/"

last_30days = pd.read_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet')


def fix_views(view: pd.Series) -> pd.Series:
    view = view.str.replace("+", "").str.replace(",", "")
    # print(view)

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if "K" in x else x)
    view = view.apply(
        lambda x: float(x.replace("M", "")) * 1000000 if "M" in str(x) else x
    )

    return view


last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]), subs=lambda x: fix_views(x["subs"])
)


last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)

['@webcentermega2359', '@CopaferOficial', '@village.construcao', '@casasaopedro8042']
0         0
1     94727
2         0
3    107327
Name: views, dtype: object
0      0
1     20
2      0
3    590
Name: subs, dtype: object
